# Alrighty, let's give Planet's Satellite imagery api a first test! 

### 1.0: Import the api key from a config file.

In [1]:
#Setup the API Key from the `PL_API_KEY` environment variable
from aFile import api_key
PLANET_API_KEY = api_key

### 1.1: Get our dependencies

In [2]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth

### 1.2: Setup the session

In [3]:
session = requests.Session()
session.auth = (PLANET_API_KEY, '')

# 2.0: Prep the data for the map(s)

### 2.1: Load the GeoJson coordinates for the map(s) we want

In [11]:
''' # REDDING RESERVOIR USED IN EXAMPLE - NOT USED
geo_json_geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        -122.52227783203125,
        40.660847697284815
      ],
      [
        -122.52227783203125,
        40.987154933797335
      ],
      [
        -122.01690673828124,
        40.987154933797335
      ],
      [
        -122.01690673828124,
        40.660847697284815
      ],
      [
        -122.52227783203125,
        40.660847697284815
      ]
    ]
  ]
}

In [38]:
''' # NYC BLOCK, CANNOT USE THIS DUE TO LIMITED LICENSE - NOT USED
geo_json_geometry = {
 "type": "Polygon",
 "coordinates": [
    [
      [
        -73.97801756858826,
        40.72943772441242
      ],
      [
        -73.98258805274963,
        40.73136457840986
      ],
      [
        -73.98582816123962,
        40.72696606629052
      ],
      [
        -73.98120403289795,
        40.725022823244004
      ],
      [
        -73.97801756858826,
        40.72943772441242
      ]
    ]
  ]
}

In [5]:
# MONTEREY CALIFORNIA AND OFFSHORE BAY
geo_json_geometry = {
 "type": "Polygon",
 "coordinates": [
    [
      [
        238.05004119873047,
        36.56783928193884
      ],
      [
        238.19526672363284,
        36.56783928193884
      ],
      [
        238.19526672363284,
        36.68824378744832
      ],
      [
        238.05004119873047,
        36.68824378744832
      ],
      [
        238.05004119873047,
        36.56783928193884
      ]
    ]
 ]
}

In [79]:
# OTHER CALIFORNIA
geo_json_geometry = {
 "type": "Polygon",
 "coordinates": [
    [
      [
        240.01110076904297,
        34.521832848280326
      ],
      [
        240.13023376464847,
        34.521832848280326
      ],
      [
        240.13023376464847,
        34.62642791261892
      ],
      [
        240.01110076904297,
        34.62642791261892
      ],
      [
        240.01110076904297,
        34.521832848280326
      ]
    ]
 ]
}

In [80]:
geo_json_geometry

{'type': 'Polygon',
 'coordinates': [[[240.01110076904297, 34.521832848280326],
   [240.13023376464847, 34.521832848280326],
   [240.13023376464847, 34.62642791261892],
   [240.01110076904297, 34.62642791261892],
   [240.01110076904297, 34.521832848280326]]]}

In [81]:
# filter for items the overlap with our chosen geometry
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geo_json_geometry
}

In [82]:
geometry_filter

{'type': 'GeometryFilter',
 'field_name': 'geometry',
 'config': {'type': 'Polygon',
  'coordinates': [[[240.01110076904297, 34.521832848280326],
    [240.13023376464847, 34.521832848280326],
    [240.13023376464847, 34.62642791261892],
    [240.01110076904297, 34.62642791261892],
    [240.01110076904297, 34.521832848280326]]]}}

In [83]:
# filter images acquired in a certain date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2018-07-01T00:00:00.000Z",
    "lte": "2018-08-01T00:00:00.000Z"
  }
}

In [84]:
date_range_filter

{'type': 'DateRangeFilter',
 'field_name': 'acquired',
 'config': {'gte': '2018-07-01T00:00:00.000Z',
  'lte': '2018-08-01T00:00:00.000Z'}}

In [85]:
# filter any images which are more than 50% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

In [86]:
cloud_cover_filter

{'type': 'RangeFilter', 'field_name': 'cloud_cover', 'config': {'lte': 0.5}}

In [87]:
# create a filter that combines our geo and date filters
# could also use an "OrFilter"
geoarea = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}


In [88]:
geoarea

{'type': 'AndFilter',
 'config': [{'type': 'GeometryFilter',
   'field_name': 'geometry',
   'config': {'type': 'Polygon',
    'coordinates': [[[240.01110076904297, 34.521832848280326],
      [240.13023376464847, 34.521832848280326],
      [240.13023376464847, 34.62642791261892],
      [240.01110076904297, 34.62642791261892],
      [240.01110076904297, 34.521832848280326]]]}},
  {'type': 'DateRangeFilter',
   'field_name': 'acquired',
   'config': {'gte': '2018-07-01T00:00:00.000Z',
    'lte': '2018-08-01T00:00:00.000Z'}},
  {'type': 'RangeFilter',
   'field_name': 'cloud_cover',
   'config': {'lte': 0.5}}]}

In [89]:
# Stats API request object
stats_endpoint_request = {
  "interval": "day",
  "item_types": ["REOrthoTile"],
  "filter": geoarea
}

In [90]:
stats_endpoint_request

{'interval': 'day',
 'item_types': ['REOrthoTile'],
 'filter': {'type': 'AndFilter',
  'config': [{'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': {'type': 'Polygon',
     'coordinates': [[[240.01110076904297, 34.521832848280326],
       [240.13023376464847, 34.521832848280326],
       [240.13023376464847, 34.62642791261892],
       [240.01110076904297, 34.62642791261892],
       [240.01110076904297, 34.521832848280326]]]}},
   {'type': 'DateRangeFilter',
    'field_name': 'acquired',
    'config': {'gte': '2018-07-01T00:00:00.000Z',
     'lte': '2018-08-01T00:00:00.000Z'}},
   {'type': 'RangeFilter',
    'field_name': 'cloud_cover',
    'config': {'lte': 0.5}}]}}

In [91]:
# Search API request object
search_endpoint_request = {
  "item_types": ["REOrthoTile"],
  "filter": geoarea
}

In [92]:
search_endpoint_request

{'item_types': ['REOrthoTile'],
 'filter': {'type': 'AndFilter',
  'config': [{'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': {'type': 'Polygon',
     'coordinates': [[[240.01110076904297, 34.521832848280326],
       [240.13023376464847, 34.521832848280326],
       [240.13023376464847, 34.62642791261892],
       [240.01110076904297, 34.62642791261892],
       [240.01110076904297, 34.521832848280326]]]}},
   {'type': 'DateRangeFilter',
    'field_name': 'acquired',
    'config': {'gte': '2018-07-01T00:00:00.000Z',
     'lte': '2018-08-01T00:00:00.000Z'}},
   {'type': 'RangeFilter',
    'field_name': 'cloud_cover',
    'config': {'lte': 0.5}}]}}

In [93]:
# fire off the POST request
result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_endpoint_request)

In [94]:
#print result.text
out_json = json.loads(result.text)
i=0
out_json

{'_links': {'_first': 'https://api.planet.com/data/v1/searches/340677faf2a946c0bda3ad0a1a76a0b0/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D',
  '_next': 'https://api.planet.com/data/v1/searches/340677faf2a946c0bda3ad0a1a76a0b0/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6ICIyMDE4LTA3LTA4VDA5OjU1OjQ4LjMzNTAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAic29ydF9kZXNjIjogdHJ1ZX0%3D',
  '_self': 'https://api.planet.com/data/v1/searches/340677faf2a946c0bda3ad0a1a76a0b0/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D'},
 'features': [{'_links':

In [95]:
for feature in out_json['features']: 
	if i == 0:
		item_id = feature['id']
	i=i+1
	print(feature['id'])

20180728_184647_1155004_RapidEye-3
20180728_184647_1155003_RapidEye-3
20180728_184647_1055026_RapidEye-3
20180724_184430_1155004_RapidEye-4
20180724_184431_1155003_RapidEye-4
20180724_184431_1055026_RapidEye-4
20180722_185002_1155003_RapidEye-2
20180722_185002_1155004_RapidEye-2
20180722_185002_1055026_RapidEye-2
20180709_184732_1155004_RapidEye-3
20180709_184732_1155003_RapidEye-3
20180709_184732_1055026_RapidEye-3
20180707_185220_1155003_RapidEye-1
20180707_185219_1155004_RapidEye-1
20180707_185220_1055026_RapidEye-1


In [96]:
# item_id = "20160707_195147_1057916_RapidEye-1"
item_id = "20180707_185220_1055026_RapidEye-1"
item_type = "REOrthoTile"
asset_type = "visual"

In [97]:
# request an item
item = \
  session.get(
    ("https://api.planet.com/data/v1/item-types/" +
    "{}/items/{}/assets/").format(item_type, item_id))

In [98]:
print(type(item))
print(item.headers)


<class 'requests.models.Response'>
{'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json', 'Date': 'Fri, 20 Sep 2019 10:02:08 GMT', 'Server': 'nginx/1.15.10', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Vary': 'Accept-Encoding', 'X-Planet-Trace-Id': 'd442fc4a-4abf-4f25-81a6-737a4906e1f1', 'Via': '1.1 edge, 1.1 google', 'Alt-Svc': 'clear', 'Transfer-Encoding': 'chunked'}


In [99]:
print(item.content)

b'{"analytic": {"_links": {"_self": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljIiwgImN0IjogIml0ZW0tdHlwZSJ9", "activate": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljIiwgImN0IjogIml0ZW0tdHlwZSJ9/activate", "type": "https://api.planet.com/data/v1/asset-types/analytic"}, "_permissions": ["download"], "md5_digest": null, "status": "inactive", "type": "analytic"}, "analytic_xml": {"_links": {"_self": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljX3htbCIsICJjdCI6ICJpdGVtLXR5cGUifQ", "activate": "https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljX3htbCIsICJjdCI6ICJpdGVtLXR5cGUifQ/activate", "type": "https://api

In [100]:
# check if the item is active, if it is inactive, it cannot be downloaded.
# (the server has to fetch the image in the background I think.)
print(item.json()['visual']['status'])

inactive


In [101]:
# let's activate the item
# Parse out useful links
links = item.json()[u"visual"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

In [102]:
links

{'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogInZpc3VhbCIsICJjdCI6ICJpdGVtLXR5cGUifQ',
 'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogInZpc3VhbCIsICJjdCI6ICJpdGVtLXR5cGUifQ/activate',
 'type': 'https://api.planet.com/data/v1/asset-types/visual'}

In [103]:
self_link

'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogInZpc3VhbCIsICJjdCI6ICJpdGVtLXR5cGUifQ'

In [104]:
activation_link

'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzA3XzE4NTIyMF8xMDU1MDI2X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogInZpc3VhbCIsICJjdCI6ICJpdGVtLXR5cGUifQ/activate'

In [105]:
# Request activation of the asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )

In [106]:
activate_result

<Response [403]>